In [1]:
# config
# === CONFIG (edit these if you like) ===
KAGGLE_URL = "https://www.kaggle.com/datasets/rodrigorosasilva/cic-ddos2019-30gb-full-dataset-csv-files"

# how much to read (keeps it light)
PER_FILE_ROWS = 150_000   # rows to read from each CSV (Colab-friendly)
MAX_ROWS_TOTAL = 600_000  # stop once we reach this many rows total

# what we’ll try as the target column (CICDDoS variants differ a bit)
TARGET_CANDIDATES = ["Label", "label", "Attack", "attack", "Class", "class", "Target", "target"]

# output
OUTPUT_CSV = "cleaned_cicddos2019_sample.csv"

In [2]:
# import libraries
!pip -q install opendatasets pandas numpy scikit-learn matplotlib seaborn

import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import opendatasets as od

from sklearn.preprocessing import LabelEncoder, StandardScaler

sns.set_theme()
pd.set_option("display.max_columns", 100)

In [3]:
#Download dataset
DATA_DIR = "/content/data"
od.download(KAGGLE_URL, data_dir=DATA_DIR)

csv_files = glob.glob(f"{DATA_DIR}/**/*.csv", recursive=True)
if not csv_files:
    csv_files = glob.glob(f"{DATA_DIR}/*.csv")

print(f"Found {len(csv_files)} CSV file(s). Showing a few:")
for p in csv_files[:5]:
    print(" •", p)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: annwangariics
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/rodrigorosasilva/cic-ddos2019-30gb-full-dataset-csv-files


100%|██████████| 3.10G/3.10G [00:45<00:00, 72.9MB/s]



Found 18 CSV file(s). Showing a few:
 • /content/data/cic-ddos2019-30gb-full-dataset-csv-files/01-12/UDPLag.csv
 • /content/data/cic-ddos2019-30gb-full-dataset-csv-files/01-12/DrDoS_SNMP.csv
 • /content/data/cic-ddos2019-30gb-full-dataset-csv-files/01-12/TFTP.csv
 • /content/data/cic-ddos2019-30gb-full-dataset-csv-files/01-12/DrDoS_SSDP.csv
 • /content/data/cic-ddos2019-30gb-full-dataset-csv-files/01-12/DrDoS_NTP.csv


In [4]:
#load a small,simple sample(straightforward concatenation)

frames = []
rows_loaded = 0

for p in csv_files:
    if rows_loaded >= MAX_ROWS_TOTAL:
        break
    # read just a slice from each file to keep memory small
    take = min(PER_FILE_ROWS, MAX_ROWS_TOTAL - rows_loaded)
    try:
        part = pd.read_csv(p, nrows=take, low_memory=False)
        frames.append(part)
        rows_loaded += len(part)
        print(f"Loaded {len(part):,} from {os.path.basename(p)}  (total: {rows_loaded:,})")
    except Exception as e:
        print("Skip (read error):", p, e)

if not frames:
    raise RuntimeError("No CSVs could be read. Check dataset structure or increase PER_FILE_ROWS.")

df = pd.concat(frames, ignore_index=True)
print("Combined shape:", df.shape)
df.head()

Loaded 150,000 from UDPLag.csv  (total: 150,000)
Loaded 150,000 from DrDoS_SNMP.csv  (total: 300,000)
Loaded 150,000 from TFTP.csv  (total: 450,000)
Loaded 150,000 from DrDoS_SSDP.csv  (total: 600,000)
Combined shape: (600000, 88)


,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,186059,172.16.0.5-192.168.50.1-58445-4463-17,172.16.0.5,58445,192.168.50.1,4463,17,2018-12-01 13:04:45.928673,1,2,0,766.0,0.0,383.0,383.0,383.0,0.0,0.0,0.0,0.0,0.0,766000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,383.0,383.0,383.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,574.5,383.0,0.0,-2,0,0,0,0,0,0,2,766,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag
1,135692,172.16.0.5-192.168.50.1-36908-9914-17,172.16.0.5,36908,192.168.50.1,9914,17,2018-12-01 13:04:45.928913,1,2,0,778.0,0.0,389.0,389.0,389.0,0.0,0.0,0.0,0.0,0.0,778000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,389.0,389.0,389.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,583.5,389.0,0.0,-2,0,0,0,0,0,0,2,778,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag
2,33822,172.16.0.5-192.168.50.1-41727-32361-17,172.16.0.5,41727,192.168.50.1,32361,17,2018-12-01 13:04:45.928915,2,2,0,750.0,0.0,375.0,375.0,375.0,0.0,0.0,0.0,0.0,0.0,375000000.0,1000000.0,2.0,0.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,1000000.0,0.0,375.0,375.0,375.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,562.5,375.0,0.0,-2,0,0,0,0,0,0,2,750,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag
3,24498,172.16.0.5-192.168.50.1-55447-5691-17,172.16.0.5,55447,192.168.50.1,5691,17,2018-12-01 13:04:45.929024,2,2,0,738.0,0.0,369.0,369.0,369.0,0.0,0.0,0.0,0.0,0.0,369000000.0,1000000.0,2.0,0.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,1000000.0,0.0,369.0,369.0,369.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,553.5,369.0,0.0,-2,0,0,0,0,0,0,2,738,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag
4,117372,172.16.0.5-192.168.50.1-58794-56335-17,172.16.0.5,58794,192.168.50.1,56335,17,2018-12-01 13:04:45.929096,1,2,0,750.0,0.0,375.0,375.0,375.0,0.0,0.0,0.0,0.0,0.0,750000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2,0,2000000.0,0.0,375.0,375.0,375.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,562.5,375.0,0.0,-2,0,0,0,0,0,0,2,750,0,0,-1,-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,UDP-lag


In [5]:
#EDA
print(df.info())
print("\nMissing (top 15):\n", df.isna().sum().sort_values(ascending=False).head(15))

# try to detect target column
target_col = None
for cand in TARGET_CANDIDATES:
    for c in df.columns:
        if c.lower() == cand.lower():
            target_col = c
            break
    if target_col:
        break

print("\nDetected target column:", target_col)

if target_col:
    print("\nClass counts:\n", df[target_col].value_counts(dropna=False))
    sns.countplot(x=df[target_col])
    plt.title("Class distribution")
    plt.xticks(rotation=45)
    plt.show()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 88 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Unnamed: 0                    600000 non-null  int64  
 1   Flow ID                       600000 non-null  object 
 2    Source IP                    600000 non-null  object 
 3    Source Port                  600000 non-null  int64  
 4    Destination IP               600000 non-null  object 
 5    Destination Port             600000 non-null  int64  
 6    Protocol                     600000 non-null  int64  
 7    Timestamp                    600000 non-null  object 
 8    Flow Duration                600000 non-null  int64  
 9    Total Fwd Packets            600000 non-null  int64  
 10   Total Backward Packets       600000 non-null  int64  
 11  Total Length of Fwd Packets   600000 non-null  float64
 12   Total Length of Bwd Packets  600000 non-nul

In [6]:
#basic cleaning
before = len(df)
df = df.drop_duplicates()
print(f"Dropped duplicates: {before - len(df)}")

# numeric → fill NaN with 0; categorical → fill NaN with "unknown"
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in df.columns if c not in num_cols]

df[num_cols] = df[num_cols].fillna(0)
for c in cat_cols:
    df[c] = df[c].astype(str).fillna("unknown").replace({"nan": "unknown"})

print("Remaining missing values:", int(df.isna().sum().sum()))

Dropped duplicates: 0
Remaining missing values: 0


In [7]:
#encode target + encode other categoricals

# target first
if target_col is not None and df[target_col].dtype == object:
    le_y = LabelEncoder()
    df[target_col] = le_y.fit_transform(df[target_col])
    print(f"Encoded target '{target_col}':", dict(zip(le_y.classes_, le_y.transform(le_y.classes_))))
else:
    print("Target already numeric or not found; no target encoding done.")

# other categorical predictors
cat_predictors = [c for c in df.columns if df[c].dtype == object and c != target_col]
for c in cat_predictors:
    df[c] = LabelEncoder().fit_transform(df[c].astype(str))

print(f"Encoded {len(cat_predictors)} categorical predictor columns.")


Target already numeric or not found; no target encoding done.
Encoded 6 categorical predictor columns.


In [9]:
#scale numeric features
scaler = StandardScaler()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

if target_col:
    X = df.drop(columns=[target_col]).copy()
    y = df[target_col].copy()
else:
    X = df.copy()
    y = None

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# Replace infinities with NaN
X = X.replace([np.inf, -np.inf], np.nan)

# Drop rows with NaN (or fill them with 0/mean if you prefer)
X = X.dropna()

# Now proceed with scaling
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

X[num_cols] = scaler.fit_transform(X[num_cols])

df_clean = X.copy()
if target_col:
    df_clean[target_col] = y

print("Cleaned shape:", df_clean.shape)
df_clean.head()

Cleaned shape: (576271, 88)


,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,0.848191,0.321354,-0.090579,1.303344,-0.023308,-1.493014,0.820905,0.022034,-0.301279,-0.015138,-0.089664,0.050717,-0.013058,0.299790,0.325685,0.315262,-0.282576,-0.037747,-0.040635,-0.04312,-0.03175,1.101494,0.948276,-0.304028,-0.302429,-0.305124,-0.005110,-0.300942,-0.309533,-0.303527,-0.304795,-0.004558,-0.045499,-0.054826,-0.055542,-0.052869,-0.087874,-0.028079,0.0,0.0,0.0,0.089217,0.002945,0.948219,-0.098065,0.327582,0.291387,0.317065,-0.190364,-0.036744,0.0,-0.009127,-0.028079,0.0,-0.814655,-0.060042,-0.049296,0.0,-0.187715,0.343518,0.315262,-0.04312,0.089217,0.0,0.0,0.0,0.0,0.0,0.0,-0.015138,0.050717,-0.089664,-0.013058,-0.69729,-0.028537,-0.040841,0.114412,-0.031274,-0.027918,-0.030738,-0.021073,-0.298102,-0.203453,-0.300943,-0.287241,-0.017739,0.088057,1.361873
1,0.267959,-1.038928,-0.090579,0.372234,-0.023308,-1.205602,0.820905,0.022040,-0.301279,-0.015138,-0.089664,0.061353,-0.013058,0.315994,0.342012,0.331560,-0.282576,-0.037747,-0.040635,-0.04312,-0.03175,1.128657,0.948276,-0.304028,-0.302429,-0.305124,-0.005110,-0.300942,-0.309533,-0.303527,-0.304795,-0.004558,-0.045499,-0.054826,-0.055542,-0.052869,-0.087874,-0.028079,0.0,0.0,0.0,0.089217,0.002945,0.948219,-0.098065,0.343946,0.307371,0.333390,-0.190364,-0.036744,0.0,-0.009127,-0.028079,0.0,-0.814655,-0.060042,-0.049296,0.0,-0.187715,0.359915,0.331560,-0.04312,0.089217,0.0,0.0,0.0,0.0,0.0,0.0,-0.015138,0.061353,-0.089664,-0.013058,-0.69729,-0.028537,-0.040841,0.114412,-0.031274,-0.027918,-0.030738,-0.021073,-0.298102,-0.203453,-0.300943,-0.287241,-0.017739,0.088057,1.361873
2,-0.905591,-0.778491,-0.090579,0.580574,-0.023308,-0.022050,0.820905,0.022045,-0.301279,-0.015138,-0.089664,0.036535,-0.013058,0.278183,0.303916,0.293530,-0.282576,-0.037747,-0.040635,-0.04312,-0.03175,0.216404,-0.150745,-0.304027,-0.302429,-0.305124,-0.005026,-0.300942,-0.309532,-0.303527,-0.304795,-0.004483,-0.045499,-0.054826,-0.055542,-0.052869,-0.087874,-0.028079,0.0,0.0,0.0,0.089217,0.002945,-0.149658,-0.098065,0.305763,0.270074,0.295299,-0.190364,-0.036744,0.0,-0.009127,-0.028079,0.0,-0.814655,-0.060042,-0.049296,0.0,-0.187715,0.321654,0.293530,-0.04312,0.089217,0.0,0.0,0.0,0.0,0.0,0.0,-0.015138,0.036535,-0.089664,-0.013058,-0.69729,-0.028537,-0.040841,0.114412,-0.031274,-0.027918,-0.030738,-0.021073,-0.298102,-0.203453,-0.300943,-0.287241,-0.017739,0.088057,1.361873
3,-1.013004,0.101823,-0.090579,1.173731,-0.023308,-1.428266,0.820905,0.022051,-0.301279,-0.015138,-0.089664,0.025899,-0.013058,0.261979,0.287589,0.277232,-0.282576,-0.037747,-0.040635,-0.04312,-0.03175,0.202822,-0.150745,-0.304027,-0.302429,-

In [10]:
#save and download the cleaned csv
df_clean.to_csv(OUTPUT_CSV, index=False)
print("Saved:", OUTPUT_CSV)

from google.colab import files
files.download(OUTPUT_CSV)


Saved: cleaned_cicddos2019_sample.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
!git config --global user.email "ann.wangari@strathmore.edu"
!git config --global user.name "annkimani-ICS"


In [15]:
!git clone https://github.com/annKimani-ICS/Random-Forest-Based-IDPS.git
%cd Random-Forest-Based-IDPS

Cloning into 'Random-Forest-Based-IDPS'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 4.74 KiB | 4.74 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/Random-Forest-Based-IDPS


In [16]:
!git fetch origin
!git checkout -b feat/s1-data-cleaning origin/feat/s1-data-cleaning

Branch 'feat/s1-data-cleaning' set up to track remote branch 'feat/s1-data-cleaning' from 'origin'.
Switched to a new branch 'feat/s1-data-cleaning'
